<a href="https://colab.research.google.com/github/garenoo/2023MLProgramming/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [38]:
titanic=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/gender_submission.csv")
train=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")
test=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test.csv")

In [39]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [40]:
# Pclass의 종류가 3가지이므로, get_dummies를 이용해서 원 핫 인코딩
Pclass_train = pd.get_dummies(train["Pclass"])
Pclass_test = pd.get_dummies(train["Pclass"])

# 각 종류별로 열 하나씩 만들어줌
Pclass_train.columns = ["pclass_1", "pclass_2", "pclass_3"]
Pclass_test.columns = ["pclass_1", "pclass_2", "pclass_3"]

# 더 이상 사용하지 않는 Pclass column은 삭제
train.drop(["Pclass"], axis = 1, inplace = True)
test.drop(["Pclass"], axis = 1, inplace = True)

# 원 핫 인코딩된 결과 추가
train = train.join(Pclass_train)
test = test.join(Pclass_test)

In [41]:
train.head()

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,pclass_1,pclass_2,pclass_3
0,1,0,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,0,0
2,3,1,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,0,0
4,5,0,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,0,1


In [42]:
# Sex의 종류가 2가지이므로, get_dummies를 이용해서 원 핫 인코딩
sex_train = pd.get_dummies(train["Sex"])
sex_test = pd.get_dummies(train["Sex"])

# 각 종류별로 열 하나씩 만들어줌
sex_train.columns = ["Female", "Male"]
sex_test.columns = ["Female", "Male"]

# 더 이상 사용하지 않는 Pclass column은 삭제
train.drop(["Sex"], axis = 1, inplace = True)
test.drop(["Sex"], axis = 1, inplace = True)

# 원 핫 인코딩된 결과 추가
train = train.join(sex_train)
test = test.join(sex_test)

In [43]:
train.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,pclass_1,pclass_2,pclass_3,Female,Male
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,0,0,1,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,1,0,0,1,0
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0,1,1,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,1,0,0,1,0
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,0,0,1,0,1


In [44]:
# Age는 비어있는 값이 있어서 그들을 평균값으로 대체
train["Age"].fillna(train["Age"].mean(), inplace = True)
test["Age"].fillna(test["Age"].mean(), inplace = True)

In [45]:
# SibSp(형제, 자매)와 Parch(부모님)은 Family라는 한 column으로 묶어서 처리
train["Family"] = train["SibSp"] + train["Parch"] + 1 # 1은 본인
test["Family"] = test["SibSp"] + test["Parch"] + 1

In [46]:
train[["Name", "Family"]].head()

,Name,Family
0,"Braund, Mr. Owen Harris",2
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",2
2,"Heikkinen, Miss. Laina",1
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",2
4,"Allen, Mr. William Henry",1


In [47]:
# test의 Fare의 빈 값 평균값으로 대체
test["Fare"].fillna(test["Fare"].mean(), inplace = True)

In [48]:
# Cabin은 null값이 너무 많아서 신뢰도가 떨어질 수 있음. 제거
train = train.drop(["Cabin"], axis = 1)
test = test.drop(["Cabin"], axis = 1)

In [49]:
# Embarked는 약간의 Null이 있는데, S에 가장 많으므로 S로 채움
train["Embarked"].fillna("S", inplace = True)
test["Embarked"].fillna("S", inplace = True)

In [50]:
# Embarked는 문자이므로 숫자로 바뀌게 원 핫 인코딩
embarked_train = pd.get_dummies(train["Embarked"])
embarked_test = pd.get_dummies(train["Embarked"])

embarked_train.columns = ["embarked_s", "embarked_c", "embarked_q"]
embarked_test.columns = ["embarked_s", "embarked_c", "embarked_q"]

train.drop(["Embarked"], axis = 1, inplace = True)
test.drop(["Embarked"], axis = 1, inplace = True)

train = train.join(embarked_train)
test = test.join(embarked_test)

In [51]:
# 필요없는 데이터 없애기
train.drop(["PassengerId", "Name", "SibSp", "Parch", "Ticket", "Male"], axis = 1, inplace = True)
test.drop(["PassengerId", "Name", "SibSp", "Parch", "Ticket", "Male"], axis = 1, inplace = True)

In [55]:
# train에서 data와 target 나누기
y = train["Survived"].values
X = train.drop("Survived", axis = 1).values

In [56]:
# test는 dataframe 형식에서 list로 변환
test_value = test.values
print(test_value)

[[34.5         7.8292      0.         ...  0.          0.
   1.        ]
 [47.          7.          1.         ...  1.          0.
   0.        ]
 [62.          9.6875      0.         ...  0.          0.
   1.        ]
 ...
 [38.5         7.25        0.         ...  0.          0.
   1.        ]
 [30.27259036  8.05        0.         ...  0.          0.
   1.        ]
 [30.27259036 22.3583      0.         ...  0.          0.
   1.        ]]


In [57]:
# train.csv 데이터셋을 X와 y로 나누고 그 안에서 또 다시 train과 test로 각각 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 201935248)

In [59]:
from sklearn.preprocessing import StandardScaler

# 표준화 (Standardization)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [60]:
from sklearn.linear_model import SGDClassifier

# 1. 데이터 로드 및 전처리
# 데이터 로드, 특성과 목표 변수 정의, 데이터 분할, 데이터 정규화 등

# 2. SGDClassifier 모델 초기화
sc = SGDClassifier(loss='log_loss', max_iter=10, random_state=201935248)

# 3. 모델 훈련
sc.fit(X_train_scaled, y_train)  # X_train_scaled: 훈련 데이터의 정규화된 특성, y_train: 훈련 데이터의 목표 변수

# 4. 모델 평가
train_accuracy = sc.score(X_train_scaled, y_train)  # 훈련 데이터에 대한 정확도
test_accuracy = sc.score(X_test_scaled, y_test)  # 테스트 데이터에 대한 정확도

print("훈련 정확도:", train_accuracy)
print("테스트 정확도:", test_accuracy)

훈련 정확도: 0.7879213483146067
테스트 정확도: 0.7374301675977654


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [62]:
from sklearn.tree import DecisionTreeClassifier

dt_classifier = DecisionTreeClassifier(max_depth=5, random_state=201935248)

dt_classifier.fit(X_train, y_train)

train_accuracy = dt_classifier.score(X_train, y_train)
test_accuracy = dt_classifier.score(X_test, y_test)

print("훈련 정확도:", train_accuracy)
print("테스트 정확도:", test_accuracy)

훈련 정확도: 0.8455056179775281
테스트 정확도: 0.770949720670391


In [63]:
# 랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=201935248)

rf_classifier.fit(X_train, y_train

train_accuracy = rf_classifier.score(X_train, y_train)
test_accuracy = rf_classifier.score(X_test, y_test)

print("훈련 정확도:", train_accuracy)
print("테스트 정확도:", test_accuracy)

훈련 정확도: 0.8637640449438202
테스트 정확도: 0.8044692737430168


In [67]:
# 히스토그램 기반 그레이디언트 부스팅
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_validate


hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, X_train, y_train, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.959974100453242 0.8259036737909977
